In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from dynamicslib.common import *
from dynamicslib.consts import *
from dynamicslib.continuation import *
from dynamicslib.targeter import *
from dynamicslib.plotting import *
import pandas as pd

In [ ]:
def xtf2X(x0, tf):
    return np.array([x0[0], x0[-2], x0[-1], tf / 2])


def X2xtf(X):
    return np.array([X[0], 0, 0, 0, X[1], X[2]]), X[-1] * 2


def stmeom2DF(eomf, stm):
    dF = np.array(
        [
            [stm[1, 0], stm[1, -2], stm[1, -1], eomf[1]],
            [stm[-3, 0], stm[-3, -2], stm[-3, -1], eomf[-3]],
            [stm[2, 0], stm[2, -2], stm[2, -1], eomf[2]],
        ]
    )
    return dF


def f_func(x0, tf, xf):
    return np.array([xf[1], xf[-3], xf[2]])


func = lambda X: f_df_CR3_single(X, X2xtf, stmeom2DF, f_func, False, muEM, 1e-10)

In [ ]:
# get the linear IC


def get_IC(Lpt, mu=muEM):
    assert Lpt in [1, 2, 3]
    xL = get_Lpts()[0, Lpt - 1]
    step = 1e-4
    # if Lpt in [2, 3]:
    #     step *= -1
    eq_soln = np.array([xL, 0, 0, 0, 0, 0])
    A = get_A(eq_soln, mu)
    evals = np.linalg.eigvals(A)
    eigmag = np.imag(evals[np.abs(np.real(evals)) < 1e-10][0])
    beta3 = (eigmag**2 + A[3, 0]) / (2 * eigmag)
    vy0 = -beta3 * eigmag * step

    x0 = np.array([xL + step, 0, 0, 0, vy0, 0])
    period = 2 * np.pi / eigmag
    X0 = xtf2X(x0, period)
    return X0

In [ ]:
Xs1 = []
e1 = []
X0, b0 = find_tangent_bif(
    get_IC(1), func, [1, 0, 0, 0], 1e-2, 1e-10, skip_changes=1, stabEps=1e-5
)
b0 *= np.sign(b0[-2])
Xs1, e1 = arclen_cont(X0, func, b0, 3e-3, 2.5, 1e-10)

In [ ]:
Xs2 = []
e2 = []
X0, b0 = find_tangent_bif(
    get_IC(2), func, [-1, 0, 0, 0], 1e-2, 1e-10, skip_changes=1, stabEps=1e-5
)
b0 *= np.sign(b0[-2])
Xs2, e2 = arclen_cont(X0, func, b0, 3e-3, 2.5, 1e-10)

In [ ]:
Xs3 = []
e3 = []
X0, b0 = find_tangent_bif(
    get_IC(3), func, [-1, 0, 0, 0], -1e-2, 1e-10, skip_changes=1, stabEps=1e-5
)
b0 *= np.sign(b0[-2])
Xs3, e3 = arclen_cont(X0, func, b0, 5e-3, 9.7, 1e-10, modified=True)

In [ ]:
es = [e1, e2, e3]
XsAll = [Xs1, Xs2, Xs3]

param_names = [
    "Index",
    "Initial x",
    "Initial z",
    "Initial vy",
    "Period",
    "Jacobi Constant",
    "Stability Index",
    "Eig1",
    "Eig2",
    "Eig3",
    "Eig4",
    "Eig5",
    "Eig6",
]

xyzs = []
data = []


for Lp in range(3):
    xyzs_lp = []
    data_lp = []
    iterator = list(enumerate(zip(XsAll[Lp], es[Lp])))
    for i, (X, evals) in tqdm(iterator):
        Xcp = X.copy()
        # xyzs_lp.append(prop_ic(Xcp, X2xtf))
        stabind = np.max([(np.abs(lam) + 1 / np.abs(lam)) / 2 for lam in evals])
        args = np.argsort(np.abs(evals))
        # eigs = evals[[*args[:2], *args[-2:]]]
        jc, period = get_JC_tf(Xcp, X2xtf)
        data_lp.append([i, *Xcp[:-1], period, jc, stabind, *evals])
    # data_lp = np.array(data_lp)
    xyzs.append(xyzs_lp)
    data.append(data_lp)

In [ ]:
df = pd.DataFrame(data[0][:525], columns=param_names).set_index("Index")
df.to_csv("L1 Axial.csv")

In [ ]:
df = pd.DataFrame(data[1][:590], columns=param_names).set_index("Index")
df.to_csv("L2 Axial.csv")

In [ ]:
df = pd.DataFrame(data[2], columns=param_names).set_index("Index")
df.to_csv("L3 Axial.csv")

In [ ]:
# plotly_family(
#     xyzs[0][:160],
#     f"L1 Axial",
#     data[0][:160],
#     param_names,
#     # html_save=f"axial_L{i+1}",
#     renderer="notebook",
# )
# plotly_family(
#     xyzs[2][::10],#[:180],
#     f"L2 Axial",
#     data[2][::10],#[:180,:-4],
#     param_names,#[:-4],
#     # html_save=f"axial_L{i+1}",
#     renderer="notebook",
# )
# plotly_family(
#     xyzs[2],
#     f"L3 Axial",
#     data[2],
#     param_names,
#     # html_save=f"axial_L{i+1}",
#     renderer="notebook",
# )

In [ ]:
len(data[0])